In [1]:
# RGB 2448x2448
# Urban land: 0,255,255: Light blue
# Agriculture land: 255,255,0: Yellow
# Rangeland: 255,0,255: purple
# Forest land: 0,255,0: light green
# Water: 0,0,255: blue
# Barren land: 255,255,255: white
# Unknown: 0,0,0: black

In [2]:
import os
from PIL import Image
import tensorflow as tf
import numpy as np
import cv2

2025-03-08 07:48:47.539285: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 07:48:47.540517: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 07:48:47.548588: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-08 07:48:47.571192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741416527.608769    9786 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741416527.61

In [3]:
image = Image.open("dataML/train/119_mask.png")
image.show()

image2 = Image.open("dataML/train/119_sat.jpg")
image2.show()

In [4]:
patch_size = 256
image_dataset = []
DataMLpath = "dataML/"

In [6]:
# Crop images 2448x2448 -> 2304x2304
#--------- NO EJECUTAR DE MOMENTO-----------
contador = 0
contador2 = 0
for root, dirs, files in os.walk(DataMLpath):
    while(contador < 3):
        path = f"{root}{dirs[contador]}"
        images = os.listdir(path)

        for i, image_name in enumerate(images):
            
            image = cv2.imread(path+"/"+image_name, 1)
            image = Image.fromarray(image)

            image = image.crop((72, 72, 2376, 2376))
            print(image_name + str(contador2))

            contador2 += 1

        contador += 1

388260_sat.jpg0
137214_sat.jpg1
180461_sat.jpg2
29694_sat.jpg3
76568_sat.jpg4
8285_sat.jpg5
482930_sat.jpg6
509523_sat.jpg7
890309_sat.jpg8
136762_sat.jpg9
551291_sat.jpg10
579026_sat.jpg11
386892_sat.jpg12
943073_sat.jpg13
687972_sat.jpg14
515578_sat.jpg15
890141_sat.jpg16
761391_sat.jpg17
842772_sat.jpg18
540125_sat.jpg19
770447_sat.jpg20
393509_sat.jpg21
525158_sat.jpg22
549550_sat.jpg23
470711_sat.jpg24
758010_sat.jpg25
457244_sat.jpg26
739567_sat.jpg27
492514_sat.jpg28
340543_sat.jpg29
541127_sat.jpg30
127801_sat.jpg31
251219_sat.jpg32
867030_sat.jpg33
480616_sat.jpg34
409953_sat.jpg35
590876_sat.jpg36
628101_sat.jpg37
128240_sat.jpg38
360483_sat.jpg39
541237_sat.jpg40
550979_sat.jpg41
443525_sat.jpg42
739296_sat.jpg43
745310_sat.jpg44
980625_sat.jpg45
553387_sat.jpg46
245649_sat.jpg47
808681_sat.jpg48
234364_sat.jpg49
499932_sat.jpg50
521804_sat.jpg51
721337_sat.jpg52
573103_sat.jpg53
212096_sat.jpg54
41687_sat.jpg55
330823_sat.jpg56
395400_sat.jpg57
647277_sat.jpg58
105036_sat.j

In [ ]:
#U-NET ARCHITECTURE
inputs = tf.keras.layers.Input((128, 128, 3))

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
#Expansive path
u6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1])
c9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)